In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [151]:
# Загрузка данных
stats = pd.read_csv('address_stats.csv')
clust = pd.read_csv('address_clust.csv')

In [166]:
stats.head(155)

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0
...,...,...,...,...,...
150,90885616,47112599,19979174,0.0,1095726.0
151,90885617,48480963,19979174,1001226.0,0.0
152,90885618,46529090,19979174,100000.0,0.0
153,90889612,48307397,19980139,0.0,74500.0


In [167]:
clust.head(155)

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1
...,...,...
150,70610601,1
151,70564583,1
152,70589431,1
153,70579434,1


In [155]:
# Объединение таблиц
all_data = pd.merge(stats, clust, on='address_id', how='outer')

# Присвоение всем внешним адресам кластера 0  
all_data.fillna(0, inplace=True)

In [156]:
all_data['cluster_id'].value_counts()

1.0    32949
2.0    25526
0.0    18249
Name: cluster_id, dtype: int64

In [158]:
all_data.head(5)

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,0.0
1,87154708,46402336,19166856,900000.0,0.0,0.0
2,87335816,46402336,19205422,0.0,900000.0,0.0
3,88124870,46402336,19374859,5500.0,0.0,0.0
4,95910795,46402336,21055878,0.0,5500.0,0.0


In [157]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76724 entries, 0 to 76723
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              76724 non-null  int64  
 1   address_id      76724 non-null  int64  
 2   transaction_id  76724 non-null  int64  
 3   received        76724 non-null  float64
 4   sent            76724 non-null  float64
 5   cluster_id      76724 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 4.1 MB


In [159]:
# Присвоение номерам кластеров типа int
all_data['cluster_id'].apply(lambda x: int(x));

In [162]:
def how_much_btc_sent(data, cluster_id_from, cluster_id_to):
  '''
  Функция принимает на вход все данные по транзакциям, номера кластеров, транзакции которых нужно посчитать.
  На выходе показывает объем транзакциq в btc между кластерами.
  '''
  # Создание отдельных таблиц для каждого кластера из общих данных
  cluster_id_from_data = data[data['cluster_id'] == cluster_id_from][['address_id','transaction_id', 'received', 'sent', 'cluster_id']]
  cluster_id_to_data = data[data['cluster_id'] == cluster_id_to][['address_id','transaction_id', 'received', 'sent', 'cluster_id']]
  
  # Объединение по транзакциям и чистка данных
  data_transactions = pd.merge(cluster_id_from_data, cluster_id_to_data, on='transaction_id', how='left')
  data_transactions.dropna(inplace=True)
  data_transactions.drop_duplicates(inplace=True)

  # Вывод результата в btc
  print('Кластер {} отправил в кластер {} : {} btc'.format(cluster_id_from, cluster_id_to, data_transactions['sent_x'].sum() / 10 ** 8))
  print('Кластер {} отправил в кластер {} : {} btc'.format(cluster_id_to, cluster_id_from, data_transactions['sent_y'].sum() / 10 ** 8))

In [163]:
# Сколько биткоинов отправил кластер 1 в кластер 2 и сколько кластер 2 отправил в кластер 1?
how_much_btc_sent(all_data, 1, 2)

Кластер 1 отправил в кластер 2 : 80.5323524 btc
Кластер 2 отправил в кластер 1 : 515.62110147 btc


In [164]:
# Сколько биткоинов отправил кластер 1 в кластер 0 и сколько кластер 0 отправил в кластер 1?
how_much_btc_sent(all_data, 1, 0)

Кластер 1 отправил в кластер 0 : 542.03460437 btc
Кластер 0 отправил в кластер 1 : 1016.06585209 btc


In [165]:
# Сколько биткоинов отправил кластер 2 в кластер 0 и сколько кластер 0 отправил в кластер 2?
how_much_btc_sent(all_data, 2, 0)

Кластер 2 отправил в кластер 0 : 3.02852 btc
Кластер 0 отправил в кластер 2 : 30.65603309 btc


In [193]:
def how_much_fee(data, cluster_id):
  '''
  Функция принимает на вход все данные по транзакциям и номер кластера.
  На выходе показывает сумму fee потраченную на обработку транзакций майнерами. 
  '''
  cluster_id_data = data[data['cluster_id'] == cluster_id]
  fee = abs(cluster_id_data['sent'].sum() - cluster_id_data['received'].sum()) / 10 ** 8

  print('Кластер {} потратил на fee : {} btc'.format(cluster_id, fee))

In [194]:
# Сколько биткоинов кластер 1 и кластер 2 потратил на fee?

how_much_fee(all_data, 1)
how_much_fee(all_data, 2)

Кластер 1 потратил на fee : 2.475371 btc
Кластер 2 потратил на fee : 2.45212861 btc
